# Lecture 83 – Docker and Containerization

## Learning Objectives
- Understand Docker containers for ML deployment
- Create Dockerfiles for ML applications
- Use docker-compose for multi-service deployment
- Optimize container images
- Deploy containerized models

## Expected Runtime
~10 minutes (includes Docker builds)

---

## 1. Why Docker for ML Deployment?

**Benefits:**
- ✓ Consistent environment across dev/staging/production
- ✓ Dependency isolation
- ✓ Easy scaling and orchestration
- ✓ Version control for entire environment
- ✓ Portability across cloud providers

In [ ]:
# Check Docker installation
!docker --version
!docker-compose --version

## 2. Review FastAPI Dockerfile

Let's examine the Dockerfile we created:

In [ ]:
!cat ../apps/fastapi_app/Dockerfile

## 3. Build Docker Image

In [ ]:
# Build the image
!cd ../apps/fastapi_app && docker build -t lecture83-fastapi:latest .

In [ ]:
# List images
!docker images | grep lecture83

## 4. Run Container

In [ ]:
# Run container with volume mount
import subprocess
import os

models_path = os.path.abspath("../models")

cmd = f"""
docker run -d \
  --name lecture83-server \
  -p 8000:8000 \
  -v {models_path}:/app/models \
  lecture83-fastapi:latest
"""

print("Running container...")
print(cmd)
# Uncomment to run:
# !{cmd}

In [ ]:
# Check running containers
!docker ps | grep lecture83

In [ ]:
# View logs
!docker logs lecture83-server --tail 20

## 5. Test Containerized API

In [ ]:
import requests

# Test health endpoint
response = requests.get("http://localhost:8000/ping")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")

## 6. Docker Compose for Multi-Service Deployment

In [ ]:
# View docker-compose.yml
!cat ../docker-compose.yml

In [ ]:
# Start all services
# Uncomment to run:
# !cd .. && docker-compose up -d

## 7. Container Optimization Tips

### Multi-stage Builds

In [ ]:
optimized_dockerfile = '''
# Multi-stage build for smaller images
FROM python:3.9-slim as builder

WORKDIR /build
COPY requirements.txt .
RUN pip install --user --no-cache-dir -r requirements.txt

# Final stage
FROM python:3.9-slim

COPY --from=builder /root/.local /root/.local
ENV PATH=/root/.local/bin:$PATH

WORKDIR /app
COPY app.py .

CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
'''

print("Optimized Dockerfile:")
print(optimized_dockerfile)

## 8. Container Management Commands

In [ ]:
commands = '''
# Container lifecycle
docker ps                          # List running containers
docker ps -a                       # List all containers
docker stop lecture83-server       # Stop container
docker start lecture83-server      # Start container
docker restart lecture83-server    # Restart container
docker rm lecture83-server         # Remove container

# Image management
docker images                      # List images
docker rmi lecture83-fastapi       # Remove image
docker image prune                 # Remove unused images

# Logs and debugging
docker logs -f lecture83-server    # Follow logs
docker exec -it lecture83-server bash  # Interactive shell
docker inspect lecture83-server    # Detailed info

# Docker Compose
docker-compose up -d               # Start services
docker-compose down                # Stop and remove
docker-compose logs -f             # Follow logs
docker-compose ps                  # List services
'''

print(commands)

## 9. Push to Container Registry

In [ ]:
registry_commands = '''
# Docker Hub
docker login
docker tag lecture83-fastapi username/lecture83-fastapi:v1.0
docker push username/lecture83-fastapi:v1.0

# AWS ECR
aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin <account>.dkr.ecr.us-east-1.amazonaws.com
docker tag lecture83-fastapi <account>.dkr.ecr.us-east-1.amazonaws.com/lecture83-fastapi:v1.0
docker push <account>.dkr.ecr.us-east-1.amazonaws.com/lecture83-fastapi:v1.0

# Google GCR
gcloud auth configure-docker
docker tag lecture83-fastapi gcr.io/<project-id>/lecture83-fastapi:v1.0
docker push gcr.io/<project-id>/lecture83-fastapi:v1.0
'''

print("Container Registry Commands:")
print(registry_commands)

## 10. Cleanup

In [ ]:
# Stop and remove container
# !docker stop lecture83-server
# !docker rm lecture83-server

print("Cleanup commands ready (uncomment to run)")

## Summary

✓ Created Dockerfile for FastAPI app  
✓ Built and ran Docker containers  
✓ Used docker-compose for multi-service deployment  
✓ Learned optimization techniques  
✓ Pushed to container registries  

### Production Checklist:
- [ ] Use slim base images
- [ ] Multi-stage builds
- [ ] .dockerignore file
- [ ] Health checks
- [ ] Resource limits
- [ ] Security scanning
- [ ] Version tagging

**Next**: `05_real_time_inference.ipynb`